In [ ]:
import pandas as pd
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

# Accuracy Stats

In [ ]:
def calc_all_stats(tp, fp, fn):
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    iou =  tp / (tp + fp + fn)
    print('precision: {}, recall: {}, f1: {}, iou: {}'.format(
        precision, recall, f1, iou))
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'iou': iou
    }

def read_process_stats(csv):
    df = pd.read_csv(csv)
    return df.sum(axis=0)

In [ ]:
all_outs = []
compare_df = read_process_stats('./out/landsat_sentinel/ls8_sentinel_2021_compare.csv')
calc_all_stats(compare_df['tp'], compare_df['fp'], compare_df['fn'])

# Count, Area, and Overlap comparisons

In [ ]:
def convert_to_array(string):
    ar = np.fromstring(string.strip('[]'),sep=' ').astype(int)
    return ar[ar!=0]

def calc_overlap_stats(df1, df2, sat_comp, y):
    df1_overlap = df1.loc[df1.overlap_count>0]
    df2_overlap = df2.loc[df2.overlap_count>0]

    df1_no_overlap = df1.loc[df1.overlap_count==0]
    df2_no_overlap = df2.loc[df2.overlap_count==0]
    out_dict = {
        'year': y,
        'satellites': sat_comp,
        # Totals
        'sat1_total_area': df1['size'].sum(),
        'sat2_total_area': df2['size'].sum(),
        'sat1_mean_size': df1['size'].mean(),
        'sat2_mean_size': df2['size'].mean(),
        'sat1_count': df1['size'].shape[0],
        'sat2_count': df2['size'].shape[0],
        # Overlap
        'sat1_overlap_total_area': df1_overlap['size'].sum(),
        'sat2_overlap_total_area': df2_overlap['size'].sum(),
        'sat1_overlap_mean_size': df1_overlap['size'].mean(),
        'sat2_overlap_mean_size': df2_overlap['size'].mean(),
        'sat1_overlap_count': df1_overlap['size'].shape[0],
        'sat2_overlap_count': df2_overlap['size'].shape[0],
        # No Overlap
        'sat1_no_overlap_total_area': df1_no_overlap['size'].sum(),
        'sat2_no_overlap_total_area': df2_no_overlap['size'].sum(),
        'sat1_no_overlap_mean_size': df1_no_overlap['size'].mean(),
        'sat2_no_overlap_mean_size': df2_no_overlap['size'].mean(),
        'sat1_no_overlap_count': df1_no_overlap['size'].shape[0],
        'sat2_no_overlap_count': df2_no_overlap['size'].shape[0],
    }
    return out_dict

In [ ]:
all_outs = []

sent_csv = './out/landsat_sentinel/sentinel_2021_v6_combined_overlaps.csv'
ls8_csv = './out/landsat_sentinel/ls8_2021_cloudfilt_v3_overlaps.csv'
df1 = pd.read_csv(sent_csv, converters={'overlaps':convert_to_array})
df2 = pd.read_csv(ls8_csv, converters={'overlaps':convert_to_array})
df1['overlap_count'] = df1['overlaps'].apply(len)
df2['overlap_count'] = df2['overlaps'].apply(len)
out_dict = calc_overlap_stats(df1, df2, sat_comp='sentinel-landsat', y=2021)

In [ ]:
pd.DataFrame(out_dict, index=[0]).to_csv('./out/landsat_sentinel/overlap_summaries.csv', index=False)

In [ ]:
overlap_stats = pd.read_csv('./out/landsat_sentinel/overlap_summaries.csv').set_index('year')

In [ ]:
print(overlap_stats)

In [ ]:
overlap_stats_sent = overlap_stats[[col for col in overlap_stats.columns if col[:4]=='sat1']]
overlap_stats_ls8 = overlap_stats[[col for col in overlap_stats.columns if col[:4]=='sat2']]
overlap_stats_sent.columns = [col[5:] for col in overlap_stats_sent.columns]
overlap_stats_ls8.columns = [col[5:] for col in overlap_stats_ls8.columns]
overlap_stats_ls8['sat'] = 'LS8'
overlap_stats_sent['sat'] = 'Sent'

In [ ]:
hatch_dict = {
    'overlap': None,
    'no_overlap': 'xxx',
}
alpha_dict = {
    'overlap': 1.0,
    'no_overlap': 0.5,
}
bar_color_dict = {'Sent': '#fc8d62',
               'LS8': '#8da0cb'}

In [ ]:
overlap_stats_both = pd.concat([overlap_stats_ls8, overlap_stats_sent]).reset_index()

In [ ]:

overlap_stats_both

In [ ]:

# build the plots
sats = ['Sent','LS8']
stacks = len(sats)  # how many stacks in each group for a tick location
x0 = np.arange(1)  # create an array of values for the ticks that can perform arithmetic with width (w)

# set the width
w = 0.35

# this needs to be adjusted based on the number of stacks; each location needs to be split into the proper number of locations
x1 = [x0 - w/stacks, x0 + w/stacks]

fig, axs = plt.subplots(1,2, figsize=(8,4))
comps = ['overlap','no_overlap']
for x, sat in zip(x1, sats):
    bottom_count = 0
    bottom_area = 0
    for comparison in comps:
        height_count = overlap_stats_both.loc[overlap_stats_both['sat']==sat, comparison + '_count'].values / 1000
        axs[0].bar(x=x, height=height_count, width=w, bottom=bottom_count,
               hatch=hatch_dict[comparison], 
               color=bar_color_dict[sat],
               alpha=alpha_dict[comparison],
               label=sat + ' ' + comparison)
        bottom_count += height_count

        height_area = overlap_stats_both.loc[overlap_stats_both['sat']==sat, comparison + '_total_area'].values / 10000
        axs[1].bar(x=x, height=height_area, width=w, bottom=bottom_area,
               hatch=hatch_dict[comparison], 
               color=bar_color_dict[sat],
               alpha=alpha_dict[comparison],
               label=sat + ' ' + comparison)
        bottom_area += height_area
for ax in axs:
       ax.set_xticks([])
       _ = ax.set_xticklabels('')
legend_patches = [
    Patch(facecolor=bar_color_dict['Sent'], label='Sentinel'),
    Patch(facecolor=bar_color_dict['LS8'],  label='Landsat 8'),
    Patch(facecolor='grey',alpha=0.2, label='Shared Reservoirs'),
    Patch(facecolor='grey',alpha=0.2, hatch='xxxx', label='Unshared Reservoirs')
]
axs[1].legend(handles=legend_patches)
axs[0].set_ylabel('Count (thousands)')
axs[1].set_ylabel('Area ($km^2$)')
axs[0].text(
    x=-0.375, y=axs[0].get_ylim()[1]*0.985,  # Relative position in axes coordinates (0,0 is bottom-left; 1,1 is top-right)
    s='(a)',          # The label text
    fontsize=10,
    fontstyle='italic',      # Font size
    verticalalignment='top',  # Align the text to the top
    horizontalalignment='left'  # Align the text to the left
)
axs[1].text(
    x=-0.375, y=axs[1].get_ylim()[1]*0.985,  # Relative position in axes coordinates (0,0 is bottom-left; 1,1 is top-right)
    s='(b)',          # The label text
    fontsize=10,      # Font size
    fontstyle='italic',      # Font size
    verticalalignment='top',  # Align the text to the top
    horizontalalignment='left'  # Align the text to the left
)
fig.tight_layout()